Simulated Annealing Demo
========================
This is a demo of simulated annealing algorithm. It is a heuristic algorithm for solving optimization problems. It is inspired by annealing in metallurgy, a technique involving heating and controlled cooling of a material to increase the size of its crystals and reduce their defects. The simulated annealing algorithm is an adaptation of the Metropolis–Hastings algorithm, a Monte Carlo method to generate sample states of a thermodynamic system, invented by the physicist Nicholas Metropolis in 1953. The simulated annealing algorithm was independently proposed by Scott Kirkpatrick, C. Daniel Gelatt Jr., and Mario P. Vecchi in 1983.

In [1]:
from calendar import c
from operator import le
from tracemalloc import stop
from turtle import color
from matplotlib.figure import Figure
import numpy as np

# if using JupyterHub, uncomment the following line
# %matplotlib nbagg

# if using VSCode, uncomment the following line
import matplotlib
matplotlib.use('TkAgg',force=True)

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.pyplot import figure

import networkx as nx


from matplotlib.axes import Axes


def next_neighbor(S: np.ndarray):
  S = S.copy()
  neighbors = []
  rows, cols = S.shape
  for i in range(rows):
    for j in range(cols):
      if S[i][j]:
        neighbors.append((i, j))
      else:
        c = 0
        for k in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
          c += k[0] >= 0 and k[0] < S.shape[0] and k[1] >= 0 and k[1] < S.shape[1] and S[k[0]][k[1]]
        if c == 0:
          neighbors.append((i, j))
  random_neighbor = neighbors[np.random.randint(len(neighbors))]
  S[random_neighbor[0]][random_neighbor[1]] = not S[random_neighbor[0]][random_neighbor[1]]
  return S

def update(frame: int, G: nx.Graph, fig: Figure, cost_vs_iteration: Axes, variations, stopped_a, costs, plots, iterations_per_frame: int = 10):
  iteration = iterations_per_frame*frame
  print("Iteration:", iteration, end='\r')
  for _ in range(iterations_per_frame):
    iteration+=1
    for stopped, cost, (name, sa) in zip(stopped_a, costs, (variations)):
      if stopped[0]:
        continue
      stop = sa.update(iteration)
      cost.append(sa.cost(sa.S))
      if stop:
        stopped[0] = True
        break
    if all(stopped[0] for stopped in stopped_a):
      # ani.event_source.stop()
      # print('stopping')
      break

  
  cost_vs_iteration.clear()
  for [stopped], plot, cost, (name, sa) in zip(stopped_a, plots, costs, variations):
    p = cost_vs_iteration.plot(cost, label=name)
    if stopped:
      cost_vs_iteration.axhline(y=cost[-1], color=p[0].get_color(), linestyle='--')
    T = sa.temperature(iteration)
    plot.clear()
    plot.set_aspect('equal')
    plot.set_title(f'T: {T:.2f} | Cost: {sa.cost(sa.S)}')
    x = sa.S.shape[0]
    y = sa.S.shape[1]
    colors = np.where(sa.S.reshape(-1), 'red', 'white')
    pos = {i: (i % x,i // y) for i in range(0, x * y)}
    drawn_nodes = nx.draw_networkx_nodes(G, pos=pos, node_color=colors, ax=plot, node_size=120)
    drawn_nodes.set_edgecolor('black')
    nx.draw_networkx_edges(G, pos=pos, ax=plot)
  cost_vs_iteration.set_xlim(0, 6000)
  cost_vs_iteration.set_ylim(0, 200)
  cost_vs_iteration.set_title('Cost vs Iteration')
  cost_vs_iteration.set_xlabel('Iteration')
  cost_vs_iteration.set_ylabel('Cost')
  cost_vs_iteration.legend()
  fig.tight_layout()

# Implement the following Simulated Annealing algorithm.

In [2]:

class SimulatedAnnealing:
  def __init__(self, S: np.ndarray, W: np.ndarray, k: float = 1., beta: float = .9, time_scale: float = .05):
    self.S = S
    self.W = W
    self.k = k
    self.time_scale = time_scale
    self.beta = beta
    # you may add more parameters if you want
    self.nothing_accepted = 0

  def update(self, t = 1):
    S_prime = next_neighbor(self.S)
    c_S = self.cost(self.S)
    c_S_prime = self.cost(S_prime)
    if self.acceptance_criteria(c_S, c_S_prime, self.temperature(t)):
      self.S = S_prime
      self.nothing_accepted = 0
    else:
      self.nothing_accepted += 1
    return self.stopping_criteria()

  def stopping_criteria(self):
    # pass # implement
    return self.nothing_accepted > 400
  
  def cost(self, S):
    return np.sum(self.W * S)

  def acceptance_criteria(self, c_S, c_S_prime, T):
    # pass # implement
    diff = c_S_prime - c_S
    if diff > 0 or np.random.rand() < np.exp(diff/(T * self.k)):
      return True
    return False
  
  def temperature(self, t: int):
    # pass # implement
    return self.beta ** (t * self.time_scale)


# Visulation 
To get a live visualization, run the next cell. 
You can create list of algorithms variants to run

In [3]:
iterations_per_frame = 10
size = 20

# Create the graph
S = np.full((size, size), False)
W = np.full((size, size), 1.)
G = nx.Graph()
G.add_nodes_from(range(0, size**2))
for i in range(0, size):
  for j in range(0, size-1):
    G.add_edge(j+size*i, (j+1)+size*i)
    G.add_edge(j*size+i, (j+1)*size+i)
    
# Pick the variations of SA to test
variations = [
  ("Fast", SimulatedAnnealing(S, W, k=1)),
  ("Slow", SimulatedAnnealing(S, W, k=20, time_scale=.01)),
]

# Create the figure
grid_size = (1+len(variations)//2, 2)
fig = plt.figure(figsize=(8, 8))
cost_vs_iteration = plt.subplot2grid(grid_size, (0, 0), colspan=2, rowspan=1, fig=fig)
plots = [
  plt.subplot2grid(grid_size, (i // 2 + 1, i % 2), colspan=1, rowspan=1, fig=fig)
  for i in range(len(variations))
]
costs = [[] for _ in variations]
stopped_a = [[False] for _ in variations]


ani = animation.FuncAnimation(fig, update, frames=600, interval=20, repeat=False, fargs=(G, fig, cost_vs_iteration, variations, stopped_a, costs, plots, iterations_per_frame))
plt.show()

stoppingn: 5530
stoppingn: 5540
stoppingn: 5550
stoppingn: 5560
stoppingn: 5570
stoppingn: 5580
stoppingn: 5590
stoppingn: 5600
stoppingn: 5610
stoppingn: 5620
stoppingn: 5630
stoppingn: 5640
stoppingn: 5650
stoppingn: 5660
stoppingn: 5670
stoppingn: 5680
stoppingn: 5690
stoppingn: 5700
stoppingn: 5710
stoppingn: 5720
stoppingn: 5730
stoppingn: 5740
stoppingn: 5750
stoppingn: 5760
stoppingn: 5770
stoppingn: 5780
stoppingn: 5790
stoppingn: 5800
stoppingn: 5810
stoppingn: 5820
stoppingn: 5830
stoppingn: 5840
stoppingn: 5850
stoppingn: 5860
stoppingn: 5870
stoppingn: 5880
stoppingn: 5890
stoppingn: 5900
stoppingn: 5910
stoppingn: 5920
stoppingn: 5930
stoppingn: 5940
stoppingn: 5950
stoppingn: 5960
stoppingn: 5970
stoppingn: 5980
stoppingn: 5990


In [4]:
# run this cell to save video
writervideo = animation.FFMpegWriter(fps=60) 
ani.save('test.mp4', writer=writervideo)

stoppingn: 0
stoppingn: 0
stoppingn: 10
stoppingn: 20
stoppingn: 30
stoppingn: 40
stoppingn: 50
stoppingn: 60
stoppingn: 70
stoppingn: 80
stoppingn: 90
stoppingn: 100
stoppingn: 110
stoppingn: 120
stoppingn: 130
stoppingn: 140
stoppingn: 150
stoppingn: 160
stoppingn: 170
stoppingn: 180
stoppingn: 190
stoppingn: 200
stoppingn: 210
stoppingn: 220
stoppingn: 230
stoppingn: 240
stoppingn: 250
stoppingn: 260
stoppingn: 270
stoppingn: 280
stoppingn: 290
stoppingn: 300
stoppingn: 310
stoppingn: 320
stoppingn: 330
stoppingn: 340
stoppingn: 350
stoppingn: 360
stoppingn: 370
stoppingn: 380
stoppingn: 390
stoppingn: 400
stoppingn: 410
stoppingn: 420
stoppingn: 430
stoppingn: 440
stoppingn: 450
stoppingn: 460
stoppingn: 470
stoppingn: 480
stoppingn: 490
stoppingn: 500
stoppingn: 510
stoppingn: 520
stoppingn: 530
stoppingn: 540
stoppingn: 550
stoppingn: 560
stoppingn: 570
stoppingn: 580
stoppingn: 590
stoppingn: 600
stoppingn: 610
stoppingn: 620
stoppingn: 630
stoppingn: 640
stoppingn: 650
stopping